## Naivni Bayesov klasifikator

In [1]:
import numpy as np

from Orange.data import Table
from Orange.data.filter import SameValue

ModuleNotFoundError: No module named 'Orange'

### Primer za ogrevanje

V letniku športne gimnazije imamo 20 učencev. Vsak od njih sodeluje pri enem od športov: ```kosarka```, ```nogomet```, ```gimnastika```. Njihovo višino smo ocenili "na oko" in vsakemu učencu pripisali eno od možnih vrednosti: ```nizek```, ```srednji``` ali ```visok```.

<img src="slike/footballers.png" width=600/>

<br/>
<font color="blue"> Kako bi novemu učencu Marku, ki je ```srednje``` rasti predlagali najprimernejši šport? </font>

In [2]:
data = Table('podatki/sportniki.tab')
print(data.domain)
print(data)

NameError: name 'Table' is not defined

Za začetek poglejmo kako popularni so posamezni športi:

In [3]:
for sport in data.domain["sport"].values:
    subset = SameValue(data.domain["sport"], sport)(data)
    
    print(sport)
    print(subset)
    print()
    
    py     = len(subset) / len(data)
    print("Sport (Y): %s, število: %d, verjetnost P(Y): %f" % (sport, len(subset), py))

NameError: name 'data' is not defined


<br/>
<br/>
Najpopularnejši šport je košarka, s katerim se ukvarja 8 oz. 40% učencev. Naš prvi predlog je torej, naj se Marko ukvarja s košarko. S tem rezultatom nismo najbolj zadovoljni, saj vidimo da med košarkaši ni veliko športnikov ```srednje``` višine. Razlog? Pri izračunu nismo upoštevali verjetnosti lastnosti oz. <i>atributa</i> o Markovi višini.
<br/>
<br/>

<div style="background-color:#00ccff; margin-left:50px; margin-right:50px"> Splošnim verjetnostmi razredov, ki smo jih izračunali pravimo <i>apriorne</i> verjetnosti.

Označimo jih z $P(Y)$, kjer je $Y$ spremenljivka razreda.
</div>

V našem primeru $Y$ zavzame vrednostmi {```kosarka```, ```nogomet```, ```gimnastika```}.



<br/>
<br/>

In [4]:
for sport in data.domain["sport"].values:
    subset_y = SameValue(data.domain["sport"],   sport)(data)
    subset_x = SameValue(data.domain["visina"], "srednji")(subset_y)
    p_xy = len(subset_x) / len(subset_y)
    
    
    
    
    print("Sport (Y): %s, št. srednje visokih: %d, verjetnost P(X=srednji|Y=%s): %f" % (sport, len(subset_x), sport, p_xy, ))
    print(subset_x)
    print()
    

NameError: name 'data' is not defined


<br/>
Zanimivo! Verjetnost ```srednje``` višine je največja med nogometaši. Ali podatek zadošča za spremembo prvotne odločitve?

<br/>
<div style="background-color:#00ccff; margin-left:50px; margin-right:50px">
Verjetnosti $P(X|Y)$ pravimo <i>pogojna verjetnost spremenljivke $X$ pri znanem $Y$</i>.  Opredeljuje verjetnost, da je v primerih razreda $Y$ atribut $X$ zavzame določeno vrednost. 
</div>

Katera verjetnost pa nas v resnici zanima? Želimo, da izračun upošteva Markovo višino in oceni verjetnost vsakega od športov. To je verjetnost

$$ P(Y|X) $$

oz. v Markovem primeru

$$ P(Y|X=srednji)$$

Za izračun te verjetnosti uporabimo

## Bayesov obrazec

Da bi izračunali verjetno razreda pri danih atributih $P(Y|X)$, potrebujemo verjetnost za vse možne kombinacije razreda $Y$ in atributov $X$, ki jo označimo z $P(X, Y)$. Iz pravil o pogojni verjetnosti sledi:

$$ P(X, Y) = P(X|Y) \cdot P(Y) = P(Y|X) \cdot P(X)$$ 

<br/>
<div style="background-color:#00ccff; margin-left:50px; margin-right:50px">
Iz česar sledi <i>Bayesov obrazec</i> za izračun $P(Y|X)$:

$$P(Y|X) = \frac{P(X|Y) \cdot P(Y)}{P(X)} $$ 
</div>
<br/>

Izračun verjetnosti razreda $Y$ pri znanih atributih $X$ je torej odvisen od apriorne verjetnosti razreda $P(Y)$, pogojne verjetnosti $P(X|Y)$ in apriorne verjetnosti atributov $P(X)$. <font color="blue">V Markovem primeru torej:</font>

$$P(Y|X=srednji) = \frac{P(X=srednji|Y) \cdot P(Y)}{P(X=srednji)} $$ 


<br/>
<br/>
Če verjetnost ocenimo za vsako možno vrednost razreda Y, torej {```kosarka```, ```nogomet```, ```gimnastika```}, dobimo odgovor na prvotno vprašanje.

In [5]:
for sport in data.domain["sport"].values:
    
    subset_y  = SameValue(data.domain["sport"],   sport)(data)        # vsi sportniki danega sporta
    subset_x  = SameValue(data.domain["visina"], "srednji")(data)     # vsi srednje visoki ucenci
    
    subset_xy = SameValue(data.domain["visina"], "srednji")(subset_y) # vsi srednje visoki ucenci v danem sportu
    
    # Izracunamo verjetnosti
    p_y  = len(subset_y)  / len(data)         
    p_x  = len(subset_x)  / len(data)
    p_xy = len(subset_xy) / len(subset_y)
    
    p_yx = (p_xy * p_y) / p_x
    
    print("Sport (Y): %s, napoved P(Y=%s | X=srednji): %f" % (sport, sport, p_yx))

NameError: name 'data' is not defined

## Implementacija Naivnega Bayesovega klasifikatorja

<i>Naivni Bayesov klasifikator</i> predpostavlja, da so atributi neodvisni med seboj, pri znanem razredu.

$$ P(Y|X_1, X_2, ..., X_p) = \frac{P(Y) \cdot P(X_1|Y) \cdot P(X_2|Y) \cdots P(X_p|Y)}{P(X)} $$

<font color="green"><b>Naredi sam/a.</b></font> Dopolni implementacijo naivnega Bayesovega klasifikatorja, ki je definiran v spodnjem odseku. Dopolniti je potrebno del kode, kjer izračunamo 
* verjetnostne porazdelitev razredov $P(Y)$
* verjetnostne porazdelitve atributov pri znanem razredu $P(X|Y)$



### Sklepanje o podatkih

V primeru diskretnih atributov lahko obe porazdelitvi dobimo s <i>preštevanjem</i>.
* $P(Y)$ <i> Kolikokrat se v podatkih pojavi razred $Y$?</i>
* $P(X|Y)$ <i> Kolikokrat se v podatkih, ki spadajo v razred $Y$, pojavi atribut $X$?</i>


<font color="blue"><b>Kaj pa $P(X)$?</b></font> To verjetnost je včasih težko izračunljiva, posebej pri visoko dimenzionalnih podatkih, saj ni nujno, da bodo v podatki prisotne vse kombinacije atributov. Na srečo ta vrednost ne vpliva na izbiro najverjetnejšega razreda za posamezen primer!

### Napovedovanje

Z nov primer $X^* = (X_1^*, X_2^*, ..., X_p^*)$ med vsemi vrednosti razreda $Y=y$, izberi tisto, ki maksimizira naslednji izraz:


$$ \text{arg max}_y \ P(Y=y) \cdot P(X_1^*|Y=y) \cdot P(X_2^*|Y=y) \cdots P(X_p^*|Y=y) $$

### Log-transformacija

Težava pri zgornjem pristopu pre praktične narave; množenje velikega števila verjetnosti hitro privede do zelo majhnih števil, ki lahko presežejo strojno natančnost. Najenostavnejša rešitev, ki vede do enake izbire razreda je naslednja 

$$ \text{arg max}_y \ \text{log } P(Y=y) + \text{log } P(X_1|Y=y) + \text{log } P(X_2|Y=y) + ... + \text{log } P(X_p|Y=y) $$

Pri implementaciji si pomagaj s podatki potnikov ladje <i><a href="https://www.kaggle.com/c/titanic">Titanic</a></i>.

Podatke najprej razdelimo na učno in tesno množico.

In [6]:
from Orange.data import Table
from numpy.random import shuffle

data = Table('titanic')
inxs = list(range(len(data)))
n = len(inxs)

shuffle(inxs)

data_training = data[inxs[:n//2]]
data_test     = data[inxs[n//2:]]

data_training.save('podatki/titanic-training.tab')
data_test.save('podatki/titanic-test.tab')

ModuleNotFoundError: No module named 'Orange'

Naložimo učne podatke in izračunamo verjetnosti.

In [7]:
data = Table('podatki/titanic-training.tab')
print(data.domain.class_var)
print(data.domain.class_var.values)



# P(X=child | Y = yes)
filt_child  = SameValue(data.domain["age"], "child")
filt_survived  = SameValue(data.domain["survived"], "yes")

p_xy = len(filt_survived(filt_child(data))) / len(filt_survived(data))
p_xy

NameError: name 'Table' is not defined

In [8]:
class NaiveBayes:
    
    
    """
    Naive Bayes classifier.
    

    :attribute self.probabilities
        Dictionary that stores
            - prior class probabilities P(Y)
            - attribute probabilities conditional on class P(X|Y)
    
    :attribute self.class_values
        All possible values of the class.
        
    :attribute self.variables
        Variables in the data. 
    
    :attribute self.trained
        Set to True after fit is called.
    """
    
    def __init__(self):
        self.trained       = False
        self.probabilities = dict()   
    
    
    def fit(self, data):
        """
        Fit a NaiveBayes classifier.
        
        :param data
            Orange data Table.        
        """
        class_variable      = data.domain.class_var    # class variable (Y) 
        self.class_values   = class_variable.values    # possible class values
        self.variables      = data.domain.variables    # all other variables (X)
        
        n = len(data) # number of all data points
        
        # Compute P(Y)
        for y in self.class_values:

            # A not too smart guess (INCORRECT)
            self.probabilities[y] = 1/len(self.class_values)
            
            # <your code here>
            # Compute class probabilities and correctly fill
            #   probabilities[y] = ... 
            # Select all examples (rows) with class = y
          
            # </your code here>
        
        # Compute P(X|Y)
        for y in self.class_values:
            
            # Select all examples (rows) with class = y
            filty = SameValue(class_variable, y)
            
            for variable in self.variables:
                for x in variable.values:
                    
                    # A not too smart guess (INCORRECT)
                    p = 1 / (len(self.variables) * len(variable.values) * len(self.class_values))
                    
                    # P(variable=x|Y=y)
                    self.probabilities[variable, x, y] = p
                    
                
                    # <your code here>
                    # Compute correct conditional class probability
                    #   probabilities[x, value, c] = ... 
                    # 
                    # Select all examples with class == y AND 
                    # variable x == value
                    # Hint: use SameValue filter twice
            
                
                    # </your code here>
    
        self.trained = True
        
    
    def predict_instance(self, row):
        """
        Predict a class value for one row.
        
        :param row
            Orange data Instance.
        :return 
            Class prediction.
        """
        curr_p = float("-inf")   # Current highest "probability" (unnormalized)
        curr_c = None            # Current most probable class
        
        for y in self.class_values:
            p = np.log(self.probabilities[y])
            for x in self.variables:
                p = p + np.log(self.probabilities[x, row[x].value, y])
            
            if p > curr_p:
                curr_p = p
                curr_c = y
                
        return curr_c, curr_p
        
   

    def predict(self, data):
        """
        Predict class labels for all rows in data.
        
        :param data
            Orange data Table.       
        :return y
            NumPy vector with predicted classes.
        """
        
        n = len(data)
        predictions = list()
        confidences = np.zeros((n, ))
        
        for i, row in enumerate(data):
            pred, cf = self.predict_instance(row)
            predictions.append(pred)
            confidences[i] = cf
    
        return predictions, confidences

Rešitev je dostopna na: rešitve/naivniBayes.ipynb

In [9]:
%run 'rešitve/naivniBayes.ipynb'

ERROR:root:File `'rešitve/naivniBayes.ipynb.py'` not found.


##  Uporaba klasifikatorja

Primer uporabe na podatkih potnikov ladje <i><a href="https://www.kaggle.com/c/titanic">Titanic</a></i>.

In [10]:
model = NaiveBayes()
model.fit(data)
model.probabilities

NameError: name 'data' is not defined

In [11]:
predictions, confidences = model.predict(data)

for row, p, c in zip(data, predictions, confidences):
    print("Row=%s, predicted class=%s confidence=%.5f" % (row, p, c))

NameError: name 'data' is not defined

## Ocenjevanje uspešnosti klasifikacije

Za ocenjevanje uspešnosti klasifikacije vsak napovedani primer primerjamo s pripadajočim resničnim razredo. Štirje možni izidi primerjave so naslednji: 

<table>
<tr>
<td>
<ul>
<li>TP: True positives (pravilno napovedani pozitivni primeri)</li>
<li>FP: False positives (napačno napovedani negativni primeri)</li>
<li>TN: True negatives (pravilno napovedani negativni primeri)</li>
<li>FN: False negatives (napačno napovedani pozitini primeri)</li>
</ul> 

<br/>
<img src="slike/type12_error.jpeg" width=400/>

</td>
<td><img width="400" src="slike/Precisionrecall.png"></img><td>
<tr/>
<table>


### Delež pravilno razvrščenih razredov (ang. classification accuracy)


$$ca = \frac{TP + TN}{TP + TN + FP + FN}$$

<font color="green">Prednosti</font>:
* Enostaven izračun, jasna interpretacija
* Uporabna mera za poljubno število razredov

<font color="red">Slabosti</font>:
* Lahko zavaja pri neuravnoteženih porazdelitvah razredov

<br/>

### Natančnost, priklic (ang. precision, recall)



$$ p = \frac{TP}{TP + FP} $$

$$ r = \frac{TP}{TP + FN} $$

<font color="green">Prednosti</font>:
* Enostaven izračun, jasna interpretacija
* Ločitev obeh tipov napak (napačno pozitivni in napačno negativni primeri)
* Uporabna tudi pri neuravnoteženih porazdelitvah razredov

<font color="red">Slabosti</font>:
* Uporabno pretežno za klasifikacijo v dva razreda
* Težko povzeti obe meri ; približek je F1-vrednost (ang. F1-score)
$$ F1 = 2 \frac{p \cdot r}{p + r} $$


<font color="green"><b>Naredi sam/a.</b></font> Napovej razrede na testni množici. Napovedane razrede primerjaj z resničnimi in izmeri klasifikacijsko točnost, natančnost, priklic in F1-vrednost.

In [12]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# uporaba metod: 
test_data      = Table('podatki/titanic-test.tab')
predictions, _ = model.predict(test_data) 
truth          = [row["survived"].value for row in test_data]
accuracy_score(truth, predictions)

NameError: name 'Table' is not defined

<font color="orange"><b>Izziv.</b></font> Nekateri atributi imajo verjetnost 0 pri posameznem razredu. Kako bi popravili klasifikator?

<font color="blue"><b>Razmisli.</b></font> Kako bi dopolnili klasifikator, če bi bili nekateri atributi lahko tudi zvezni? Namig: spomni se vaj, ko smo spoznali <i>verjetnostne porazdelitve</i> zveznih spremenljivk. 